## Imports

In [ ]:
%pip install torch_geometric

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
import torch.nn.functional as F
import networkx as nx
from torch_geometric.utils import to_dense_adj
from torch_geometric.datasets import TUDataset, Planetoid
from time import perf_counter
import matplotlib.pyplot as plt
import random
import numpy as np
from sklearn.manifold import TSNE


# for reproduceability
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

## 1) Graph Classification with Different Pooling Mechanisms
In this section, we are going to work on graph classification using GCN with different pooling mechanisms. As you know, GCNs modify the node feature matrix $X \in \mathbb{R}^{N\times d}$ by updating the embeddings of each node in the graph.

For graph classification, we need to get a graph-level embedding vector $d \in \mathbb{R}^{d \times 1}$. To do so, we need to **pool** node embedding vectors into a single vector.

We are going look at different pooling mechanisms, namely:
- Global Mean Pooling
- Global Max Pooling
- Global Sum Pooling

![gc.jpg](figures/gc.jpg)


### 1.1) Dataset
The most common task for graph classification is **molecular property prediction**, in which molecules are represented as graphs, and the task may be to infer whether a molecule inhibits HIV virus replication or not.

The TU Dortmund University has collected a wide range of different graph classification datasets, known as the [**TUDatasets**](https://chrsmrrs.github.io/datasets/), which are also accessible via [`torch_geometric.datasets.TUDataset`](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch_geometric.datasets.TUDataset) in PyTorch Geometric.
Let's load and inspect one of the smaller ones, the **MUTAG dataset**:

In [ ]:
mutag_dataset = TUDataset(root='data/TUDataset', name='MUTAG')

#### 1.1.1) Dataset Statistics
Let's see the statistics of the dataset.

In [ ]:
print()
print(f'Dataset: {mutag_dataset}:')
print('====================')
print(f'Number of graphs: {len(mutag_dataset)}')
print(f'Number of features: {mutag_dataset.num_features}')
print(f'Number of classes: {mutag_dataset.num_classes}')

data = mutag_dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

#### 1.1.2) Train-Test Split and Preprocessing
PyG keeps the adj matrix in **COO format**, we convert it to $N\times N$ adj matrix format using `to_dense_adj` utility function.


In [ ]:
def preprocess_gc_dataset(dataset):
    train_ds, test_ds = train_test_split(dataset, train_size=0.8, shuffle=True, random_state=seed)

    # PyG keeps the adj matrix in COO format, we convert it to NxN adj matrix format using to_dense_adj utility function
    train_ds = [(to_dense_adj(graph.edge_index).squeeze(), graph.x, graph.y) for graph in train_ds]
    test_ds = [(to_dense_adj(graph.edge_index).squeeze(), graph.x, graph.y) for graph in test_ds]

    return train_ds, test_ds

train_ds, test_ds = preprocess_gc_dataset(mutag_dataset)

print(f"Train dataset size: {len(train_ds)}")
print(f"Test dataset size: {len(test_ds)}")

#### 1.1.3) Visualization of Graphs
Let's visualize one of the graphs using `networkx` library.

In [ ]:
# MUTAG graphs are undirected,
# so set to_undirected=True to visualize it as an undirected graph (default will be to visualize as directed graph)
nx_graph = nx.from_numpy_array(train_ds[0][0].numpy())
nx.draw(nx_graph)

### 1.2) GCN Model

Remember from the Tutorial 1 that the feature update rule for the next layer $ H_{k+1} $ in a graph convolutional network is given by the equation

$$
H_{k+1} = \sigma(\tilde{D}^{-1/2} \tilde{A} \tilde{D}^{-1/2} H_k \Omega_k + \beta)
$$

where  $\tilde{D}$ is the degree matrix with added self-loops, $\tilde{A}$ is the adjacency matrix with self-loops, $H_k$ are the features from the previous layer, $ \Omega_k $ is the weight matrix at layer $ k $, $\beta$ is the bias term, and $\sigma $ denotes the activation function.

In [ ]:
class GCNLayer(nn.Module):
    """
    A single layer of a Graph Convolutional Network (GCN).
    ...
    """
    def __init__(self, input_dim, output_dim):
        super(GCNLayer, self).__init__()
        self.Omega = nn.Parameter(torch.randn(input_dim, output_dim) * torch.sqrt(torch.tensor(2.0) / (input_dim + output_dim)))
        self.beta = nn.Parameter(torch.zeros(output_dim))

    def forward(self, H_k, A_normalized):
        agg = torch.matmul(A_normalized, H_k)
        H_k_next = torch.matmul(agg, self.Omega) + self.beta
        return H_k_next

class GCN_GC(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, pooling_fn):
        super(GCN_GC, self).__init__()

        # Define GCN layers
        self.conv1 = GCNLayer(input_dim, hidden_dim)
        self.conv2 = GCNLayer(hidden_dim, num_classes)

        self.pooling_fn = pooling_fn

    def forward(self, A, X):
        # Apply self-connections
        A = A + torch.eye(A.size(0))

        # Apply normalization
        epsilon = 1e-5  # Small constant to avoid division by zero
        d = A.sum(1) + epsilon
        D_inv = torch.diag(torch.pow(d, -0.5))
        A = D_inv @ A @ D_inv

        # Pass through GCN layers
        H1 = self.conv1(X, A)
        H1 = F.relu(H1)
        H2 = self.conv2(H1, A)

        pooled = self.pooling_fn(H2)

        return pooled

### 1.3) Global Mean Pooling
![mean_pool.jpg](figures/mean_pool.jpg)

In [ ]:
def global_mean_pooling(X: torch.Tensor):
    """
    Params:
        X (tensor): Node feature matrix of shape (Nxd)
    Returns:
        Combined embeddings vector of shape (num_classes,)
    """

    return X.mean(dim=0)

### 1.4) Global Max Pooling
![max_pool.jpg](figures/max_pool.jpg)


In [ ]:
def global_max_pooling(X: torch.Tensor):
    """
    Params:
        X (tensor): Node feature matrix of shape (Nxd)
    Returns:
        Combined embeddings vector of shape (num_classes,)
    """

    return X.max(dim=0).values

### 1.5) Global Sum Pooling
![sum_pool.jpg](figures/sum_pool.jpg)


In [ ]:
def global_sum_pooling(X: torch.Tensor):
    """
    Params:
        X (tensor): Node feature matrix of shape (Nxd)
    Returns:
        Combined embeddings vector of shape (num_classes,)
    """

    return X.sum(dim=0)

### 1.6) Training and Testing

In [ ]:
def train_epoch_gc(model, dataset, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for A, X, y in dataset:
        optimizer.zero_grad()
        output = model(A, X)

        # Compute loss
        loss = loss_fn(output.view(1, -1), y)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataset)

    return average_loss


def train_gc(model, train_ds, optimizer, loss_fn, num_epochs):
    loss_values = []

    # Training loop
    for epoch in range(num_epochs):
        loss = train_epoch_gc(model, train_ds, optimizer, loss_fn)

        print(f"Epoch {epoch+1}, Loss: {loss}")
        loss_values.append(loss)

    return loss_values

@torch.no_grad()
def test_gc(model, dataset):
    model.eval()
    true_labels = []
    predicted_labels = []

    for A, X, y in dataset:
        output = model(A, X)

        predicted = output.argmax()

        true_labels.append(y.cpu().item())
        predicted_labels.append(predicted.cpu().item())

    # Debug: Print true and predicted labels
    print("True labels:", true_labels)
    print("Predicted labels:", predicted_labels)

    # Calculate accuracy, precision, sensitivity (recall), and F1-score
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average='macro')
    sensitivity = recall_score(true_labels, predicted_labels, average='macro')
    f1 = f1_score(true_labels, predicted_labels, average='macro')

    # Print out the evaluation metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Sensitivity: {sensitivity:.4f}")
    print(f"F1-score: {f1:.4f}")

    return accuracy, precision, sensitivity, f1


In [ ]:
configurations = {
    "Global Mean Pooling": global_mean_pooling,
    "Global Max Pooling": global_max_pooling,
    "Global Sum Pooling": global_sum_pooling,
}

num_hidden = 64
num_epochs = 30
learning_rate = 0.001

evaluation_metrics = {}
train_loss_values = {}

colors = ["red", "green", "blue"]

def train_and_test_gc(pooling_name, pooling_fn):
    print(f"\n==== {pooling_name} ====\n")

    model = GCN_GC(mutag_dataset.num_features, num_hidden, mutag_dataset.num_classes, pooling_fn)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

    print("\nTraining:\n")
    train_loss_values[pooling_name] = train_gc(model, train_ds, optimizer, loss_fn, num_epochs)

    print("\nTesting:\n")
    evaluation_metrics[pooling_name] = test_gc(model, test_ds)

# Train and test for each pooling configuration
for pooling_name, pooling_fn in configurations.items():
    train_and_test_gc(pooling_name, pooling_fn)

# Plot training loss values
plt.figure(figsize=(10, 6))
for i, (pooling_name, loss_values) in enumerate(train_loss_values.items()):
    plt.plot(range(1, num_epochs + 1), loss_values, label=pooling_name, linewidth=4, color=colors[i])

plt.title('Training Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

fig, axes = plt.subplots(1, 4, figsize=(24, 6))
metrics_names = ['Accuracy', 'Precision', 'Sensitivity', 'F1-score']

for i, metric_name in enumerate(metrics_names):
    axes[i].bar(
        configurations.keys(),
        [evaluation_metrics[key][i] for key in configurations],
        color=['red', 'green', 'blue'],
        alpha=0.7
    )

    axes[i].set_title(metric_name)
    axes[i].set_ylabel('Score')
    axes[i].set_xticklabels(configurations.keys(), rotation=45, ha='right')
    axes[i].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


### 1.7) Adding a Learnable Classification Head
Let's see what happens if we add a **fully-connected classification layer** on top of the convolution layers and pooling.



In [ ]:
class GCN_GC_With_Classification_Head(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, pooling_fn):
        super(GCN_GC_With_Classification_Head, self).__init__()

        # Define GCN layers
        self.conv1 = GCNLayer(input_dim, hidden_dim)
        self.conv2 = GCNLayer(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, num_classes)

        self.pooling_fn = pooling_fn

    def forward(self, A, X):
        # Apply self-connections
        A = A + torch.eye(A.size(0))

        # Apply normalization
        epsilon = 1e-5  # Small constant to avoid division by zero
        d = A.sum(1) + epsilon
        D_inv = torch.diag(torch.pow(d, -0.5))
        A = D_inv @ A @ D_inv

        # Pass through GCN layers
        H1 = self.conv1(X, A)
        H1 = F.relu(H1)
        H2 = self.conv2(H1, A)

        pooled = self.pooling_fn(H2)
        fc_out = self.fc(pooled)

        return fc_out

In [ ]:
configurations = {
    "Global Mean Pooling": (global_mean_pooling, False),
    "Global Mean Pooling + Linear Layer": (global_mean_pooling, True),
    "Global Max Pooling": (global_max_pooling, False),
    "Global Max Pooling + Linear Layer": (global_max_pooling, True),
    "Global Sum Pooling": (global_sum_pooling, False),
    "Global Sum Pooling + Linear Layer": (global_sum_pooling, True),
}

num_hidden = 64
num_epochs = 30
learning_rate = 0.001

evaluation_metrics = {}
train_loss_values = {}

colors = ["red", "purple", "blue", "cyan", "green", "lime"]

def train_and_test_gc_with_lin(pooling_name, pooling_fn, linear_layer):
    print(f"\n==== {pooling_name} ====\n")

    if linear_layer:
        model = GCN_GC_With_Classification_Head(mutag_dataset.num_features, num_hidden, mutag_dataset.num_classes, pooling_fn)
    else:
        model = GCN_GC(mutag_dataset.num_features, num_hidden, mutag_dataset.num_classes, pooling_fn)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

    print("\nTraining:\n")
    train_loss_values[pooling_name] = train_gc(model, train_ds, optimizer, loss_fn, num_epochs)

    print("\nTesting:\n")
    evaluation_metrics[pooling_name] = test_gc(model, test_ds)

# Train and test for each pooling configuration
for pooling_name, (pooling_fn, linear_layer) in configurations.items():
    train_and_test_gc_with_lin(pooling_name, pooling_fn, linear_layer)

# Plot training loss values
plt.figure(figsize=(10, 6))
for i, (pooling_name, loss_values) in enumerate(train_loss_values.items()):
    plt.plot(range(1, num_epochs + 1), loss_values, label=pooling_name, linewidth=4, color=colors[i])

plt.title('Training Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

fig, axes = plt.subplots(1, 4, figsize=(24, 6))
metrics_names = ['Accuracy', 'Precision', 'Sensitivity', 'F1-score']

for i, metric_name in enumerate(metrics_names):
    axes[i].bar(
        configurations.keys(),
        [evaluation_metrics[key][i] for key in configurations],
        color=colors,
        alpha=0.7
    )

    axes[i].set_title(metric_name)
    axes[i].set_ylabel('Score')
    axes[i].set_xticklabels(configurations.keys(), rotation=45, ha='right')
    axes[i].grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


## 2) Simple Graph Convolution (SGC)
In this section, we are going to implement Simple Graph Convolution (SGC) model and compare it with GCN on a node-wise classification task.

Simple Graph Convolution (SGC) was introduced by [Wu et al. (2019)](https://arxiv.org/pdf/1902.07153.pdf) as a **scalable** graph convolution architecture. They reduce the excess complexity of GCNs by repeatedly removing the nonlinearities between GCN layers and collapsing the resulting function into a **single linear transformation**.

![sgc.jpg](figures/sgc.jpg)

They hypothesize that the nonlinearity between GCN layers is not critical - but that the majority of the benefit arises from the local averaging. They therefore remove the nonlinear transition functions between each layer and only keep the final softmax (in order to obtain probabilistic outputs). The resulting model is linear, but still has the same increased “receptive field” of a K-layer GCN

$$\hat{\mathbf{Y}} = softmax(\mathbf{S}...\mathbf{S}\mathbf{S}\mathbf{X}\mathbf{\Theta}^{(1)}\mathbf{\Theta}^{(2)}...\mathbf{\Theta}^{(K)})$$

where $$\mathbf{S} = \tilde{D}^{-1/2} \tilde{A} \tilde{D}^{-1/2}$$

To simplify notation, one can collapse the repeated multiplication with the normalized adjacency matrix S into a single matrix by raising S to the K-th power. Also, one can reparameterize the weights into a single matrix $\mathbf{\Theta}=\mathbf{\Theta}^{(1)}\mathbf{\Theta}^{(2)}...\mathbf{\Theta}^{(K)}$. The resulting classifier becomes

$$\hat{\mathbf{Y}}_{SGC} = softmax(\mathbf{S}^K\mathbf{X}\mathbf{\Theta})$$

Furthermore, computation of $\mathbf{S}^K\mathbf{X}$ requires no weight, it is essentially equivalent to a feature
pre-processing step and the entire training of the model reduces to straight-forward multi-class logistic regression on the pre-processed features. In short, we can **precompute** the neighborhood propagation matrix $\mathbf{S}^K\mathbf{X}$ and training of SGC simplifies into a **logistic regression training**.





### 2.1) Implementation of SGC

In [ ]:
class SGC(nn.Module):
    """
    A Simple PyTorch Implementation of Logistic Regression.
    Assuming the features have been preprocessed with k-step graph propagation.
    """
    def __init__(self, input_dim, num_classes):
        super(SGC, self).__init__()

        self.W = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.W(x)

### 2.2) Precomputation of Propagation

In [ ]:
def sgc_precompute(X, S, deg):
    """
        Precomputes the k-step (=deg=degree) graph propagation.
        Params:
            X (tensor): Node feature matrix of shape Nxd
            S (tensor): Normalized and self-looped adj. matrix of shape NxN
            deg (int): Degree (or k) of propagation (k-step propagation)

        Returns:
            Preprocessed features matrix of shape Nxd
    """

    t = perf_counter()
    for _ in range(deg):
        X = torch.mm(S, X)
    precompute_time = perf_counter()-t
    return X, precompute_time

### 2.3) Baseline GCN Implementation for Node Classifcation (NC)

In [ ]:
class GCN_NC(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(GCN_NC, self).__init__()

        # Define GCN layers
        self.conv1 = GCNLayer(input_dim, hidden_dim)
        self.conv2 = GCNLayer(hidden_dim, num_classes)

    def forward(self, A, X):
        H1 = self.conv1(X, A)
        H1 = F.relu(H1)
        H2 = self.conv2(H1, A)

        return H2

### 2.4) Cora Dataset

To demonstrate, we make use of the `Cora` dataset, which is a **citation network** where nodes represent documents.
Each node is described by a 1433-dimensional bag-of-words feature vector.
Two documents are connected if there exists a citation link between them.
The task is to infer the category of each document (7 in total).

This dataset was first introduced by [Yang et al. (2016)](https://arxiv.org/abs/1603.08861) as one of the datasets of the `Planetoid` benchmark suite.
We again can make use [PyTorch Geometric](https://github.com/rusty1s/pytorch_geometric) for an easy access to this dataset via [`torch_geometric.datasets.Planetoid`](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html#torch_geometric.datasets.Planetoid):

In [ ]:
cora_dataset = Planetoid(root='data/Planetoid', name='Cora')

In [ ]:
def print_nc_dataset_stats(dataset):
    print()
    print(f'Dataset: {dataset}:')
    print('======================')
    print(f'Number of graphs: {len(dataset)}')
    print(f'Number of features: {dataset.num_features}')
    print(f'Number of classes: {dataset.num_classes}')

    data = dataset[0]  # Get the first graph object.

    print()
    print(data)
    print('===========================================================================================================')

    # Gather some statistics about the graph.
    print(f'Number of nodes: {data.num_nodes}')
    print(f'Number of edges: {data.num_edges}')
    print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
    print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
    print(f'Has isolated nodes: {data.has_isolated_nodes()}')
    print(f'Has self-loops: {data.has_self_loops()}')
    print(f'Is undirected: {data.is_undirected()}')

print_nc_dataset_stats(cora_dataset)

In [ ]:
def preprocess_nc_dataset(dataset):
    g = dataset[0]

    # Extract features, labels
    features = g.x
    labels = g.y
    A = to_dense_adj(g.edge_index).squeeze()

    # Apply self-connections
    A = A + torch.eye(A.size(0))

    # Apply normalization
    epsilon = 1e-5  # Small constant to avoid division by zero
    d = A.sum(1) + epsilon
    D_inv = torch.diag(torch.pow(d, -0.5))
    A = D_inv @ A @ D_inv

    num_nodes = g.num_nodes
    num_train = int(0.8 * num_nodes)  # 80% for training
    num_test = int(0.2 * num_nodes)    # 20% for testing

    print(f"Number of training nodes: {num_train}")
    print(f"Number of testing nodes: {num_test}")

    # Create a random permutation of node indices
    indices = torch.randperm(num_nodes)

    # Assign the first num_train nodes to the training set
    # Assign the remaining nodes to the test set
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    train_mask[indices[:num_train]] = True
    test_mask[indices[:num_train]] = True

    train_features = features[train_mask]
    test_features = features[test_mask]

    train_labels = labels[train_mask]
    test_labels = labels[test_mask]

    train_adj = A[train_mask][:, train_mask]
    test_adj = A[test_mask][:, test_mask]

    return train_features, test_features, train_labels, test_labels, train_adj, test_adj

cora_train_X, cora_test_X, cora_train_y, cora_test_y, cora_train_A, cora_test_A = preprocess_nc_dataset(cora_dataset)

### 2.5) Experiment Setup

In [ ]:
num_feats = cora_dataset.num_features
num_classes = cora_dataset.num_classes
hidden_dim = 64
sgc_deg = 2
num_epochs = 300
lr = 0.2
sgc_loss_values = []
gcn_loss_values = []

sgc_epoch_times = []
gcn_epoch_times = []

sgc = SGC(num_feats, num_classes)
gcn = GCN_NC(num_feats, hidden_dim, num_classes)

loss_fn = nn.CrossEntropyLoss()

sgc_optimizer = optim.AdamW(sgc.parameters(), lr=lr)
gcn_optimizer = optim.AdamW(gcn.parameters(), lr=lr)


### 2.6) Precomputing Neighborhood Propagation

In [ ]:
cora_train_X_precomputed, precomputing_time = sgc_precompute(cora_train_X, cora_train_A, sgc_deg)
print(f"Precomputation time: {precomputing_time:.4f} seconds")

### 2.7) Training of SGC

In [ ]:
print("=== TRAINING SGC ===")

# Training loop
for epoch in range(num_epochs):
    # Record the start time of the epoch
    epoch_start_time = perf_counter()

    sgc.train()
    sgc_optimizer.zero_grad()
    output = sgc(cora_train_X_precomputed)

    # Compute loss
    loss = loss_fn(output, cora_train_y)

    # Backward pass and optimization
    loss.backward()
    sgc_optimizer.step()

    # Record the end time of the epoch
    epoch_end_time = perf_counter()

    # Calculate the time taken for the epoch
    epoch_time = epoch_end_time - epoch_start_time

    # Store the epoch time in the list
    sgc_epoch_times.append(epoch_time)

    print(f"Epoch {epoch+1}, Loss: {loss:.4f}, Time: {epoch_time:.4f} seconds")
    sgc_loss_values.append(loss)

# Calculate the average time per epoch
sgc_avg_time_per_epoch = sum(sgc_epoch_times) / len(sgc_epoch_times)
sgc_training_time = sum(sgc_epoch_times)

### 2.8) Testing of SGC

In [ ]:
cora_test_X_precomputed, time = sgc_precompute(cora_test_X, cora_test_A, sgc_deg)

In [ ]:
with torch.no_grad():
    sgc.eval()
    output = sgc(cora_test_X_precomputed)

    _, predicted = torch.max(output, 1)

    correct = (predicted == cora_test_y).sum().item()
    total = cora_test_y.size(0)
    sgc_accuracy = correct / total

### 2.9) Training of GCN

In [ ]:
print("=== TRAINING GCN ===")

# Training loop
for epoch in range(num_epochs):
    # Record the start time of the epoch
    epoch_start_time = perf_counter()

    gcn.train()
    gcn.zero_grad()
    output = gcn(cora_train_A, cora_train_X)

    # Compute loss
    loss = loss_fn(output, cora_train_y)

    # Backward pass and optimization
    loss.backward()
    gcn_optimizer.step()

    # Record the end time of the epoch
    epoch_end_time = perf_counter()

    # Calculate the time taken for the epoch
    epoch_time = epoch_end_time - epoch_start_time

    # Store the epoch time in the list
    gcn_epoch_times.append(epoch_time)

    print(f"Epoch {epoch+1}, Loss: {loss:.4f}, Time: {epoch_time:.4f} seconds")
    gcn_loss_values.append(loss)

# Calculate the average time per epoch
gcn_avg_time_per_epoch = sum(gcn_epoch_times) / len(gcn_epoch_times)
gcn_training_time = sum(gcn_epoch_times)

### 2.10) Testing of GCN

In [ ]:
with torch.no_grad():
    gcn.eval()
    output = gcn(cora_test_A, cora_test_X)

    _, predicted = torch.max(output, 1)

    correct = (predicted == cora_test_y).sum().item()
    total = cora_test_y.size(0)
    gcn_accuracy = correct / total


### 2.11) Comparison

In [ ]:
# Values for plotting
avg_time_per_epoch = [sgc_avg_time_per_epoch, gcn_avg_time_per_epoch]
total_training_time = [sgc_training_time, gcn_training_time]
accuracy = [sgc_accuracy, gcn_accuracy]

# Labels for the bars
labels = ['SGC', 'GCN']

# Plotting average time per epoch
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.bar(labels, avg_time_per_epoch, color=['blue', 'green'], alpha=.7)
plt.title('Average Time per Epoch')
plt.ylabel('Time (seconds)')

# Plotting total training time
plt.subplot(1, 3, 2)
plt.bar(labels, total_training_time, color=['blue', 'green'], alpha=.7)
plt.title('Total Training Time')
plt.ylabel('Time (seconds)')

# Plotting test accuracy
plt.subplot(1, 3, 3)
plt.bar(labels, accuracy, color=['blue', 'green'], alpha=.7)
plt.title('Test Accuracy')
plt.ylabel('Accuracy')

plt.tight_layout()
plt.show()

print()
print(f"Average time per Epoch for SGC: {sgc_avg_time_per_epoch:.4f} seconds")
print(f"Average time per Epoch for SGC: {gcn_avg_time_per_epoch:.4f} seconds")
print()
print(f"Total training time for SGC: {sgc_training_time:.4f} seconds")
print(f"Total training time for SGC (including precomputation): {sgc_training_time + precomputing_time:.4f} seconds")
print(f"Total training time for GCN: {gcn_training_time:.4f} seconds")
print()
print(f"SGC Test Acc: {sgc_accuracy}")
print(f"GCN Test Acc: {gcn_accuracy}")

### 2.12) Visualization of Node Embeddings

For visualization, we make use of [**TSNE**](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) to embed our 7-dimensional node embeddings onto a 2D plane.

In [ ]:
def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

Let's visualize the node embeddings of our **untrained** GCN network.

In [ ]:
untrained_gcn = GCN_NC(num_feats, hidden_dim, num_classes)

# Use the untrained model
random_preds = untrained_gcn(cora_test_A, cora_test_X)

visualize(random_preds, cora_test_y)

Now, let's use the GCN that we **trained** in section 2.9

In [ ]:
trained_gcn = gcn

# Use the trained model
preds = trained_gcn(cora_test_A, cora_test_X)

visualize(preds, cora_test_y)

## 3) Attention Aggregation for GCNs (Exercise)
As you know, there are different aggrgetation methods for GCNs. Until know, we only used Kipf normalization in the form of $\tilde{D}^{-1/2} \tilde{A} \tilde{D}^{-1/2}$.

In this part, you are going to implement **attention based aggregation**.

First, you need to apply dot product attention.

<img src="figures/3_1.jpg" width="400" />


Then, you need to implement the following graph attention network

<img src="figures/3_2.jpg" width="400" />

You can visit the [Lecture 3.5 Global and local aggregation methods](https://www.youtube.com/watch?v=zRmzVkidkqA).

You can use the Cora dataset.

In [ ]:
class GCN_NC(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(GCN_NC, self).__init__()

        # Define GCN layers
        self.conv1 = GCNLayer(input_dim, hidden_dim)
        self.conv2 = GCNLayer(hidden_dim, num_classes)

    def forward(self, A, X):
        H1 = self.conv1(X, A)
        H1 = F.relu(H1)
        H2 = self.conv2(H1, A)

        return H2

In [ ]:
print("=== TRAINING GCN ===")

# Training loop
for epoch in range(num_epochs):
    # Record the start time of the epoch
    epoch_start_time = perf_counter()

    gcn.train()
    gcn.zero_grad()
    output = gcn(cora_train_A, cora_train_X)

    # Compute loss
    loss = loss_fn(output, cora_train_y)

    # Backward pass and optimization
    loss.backward()
    gcn_optimizer.step()

    # Record the end time of the epoch
    epoch_end_time = perf_counter()

    # Calculate the time taken for the epoch
    epoch_time = epoch_end_time - epoch_start_time

    # Store the epoch time in the list
    gcn_epoch_times.append(epoch_time)

    print(f"Epoch {epoch+1}, Loss: {loss:.4f}, Time: {epoch_time:.4f} seconds")
    gcn_loss_values.append(loss)

# Calculate the average time per epoch
gcn_avg_time_per_epoch = sum(gcn_epoch_times) / len(gcn_epoch_times)
gcn_training_time = sum(gcn_epoch_times)